# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [ ]:
!pip install openai==1.2 tiktoken datasets session-info --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages t

### Imports

In [ ]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [ ]:
session_info.show(excludes=['backports']) #Excluding backports for session info funtion workd so used this command

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [ ]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [ ]:
chat_model_id = creds["CHATGPT_MODEL"]

In [ ]:
# Test the API client --- Testing API working or Not working in this step
test_prompt = [
    {"role": "system", "content": "Say 'Hello, world!'"}
]

try:
    response = client.chat.completions.create(
        model=chat_model_id,
        messages=test_prompt,
        temperature=0,
        max_tokens=5
    )
    print("API Client Test Response:", response.choices[0].message.content.strip())
except Exception as e:
    print(f"API Client Test Failed: {e}")

API Client Test Response: Hello, world!


### Utilities

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

**(A) Upload and read csv file (2 Marks)**

In [ ]:
 cs_reviews_df = pd.read_csv("courier-service_reviews.csv")
# Read CSV File Here

In [ ]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [ ]:
cs_reviews_df.sample(15)

,id,review,sentiment
23,24,The tracking system used by ExpressWay Logisti...,Negative
72,73,I just wanted to take a moment to thank Expres...,Positive
110,111,I was promised to be updated with courier part...,Negative
7,8,ExpressWay Logistics' refusal to take responsi...,Negative
55,56,ExpressWay Logistics offers a convenient onlin...,Negative
39,40,Moving across the country is undoubtedly a str...,Positive
17,18,ExpressWay Logistics could enhance their value...,Negative
42,43,I encountered numerous issues with ExpressWay ...,Negative
77,78,ExpressWay Logistics is unreliable and untrust...,Negative
63,64,ExpressWay Logistics promises efficient delive...,Negative


In [ ]:
cs_reviews_df.sample(5)

,id,review,sentiment
38,39,I had a frustrating experience with ExpressWay...,Negative
118,119,I had a last-minute change to my shipping addr...,Positive
43,44,ExpressWay Logistics' advanced analytics tools...,Positive
114,115,My experience with ExpressWay Logistics was ma...,Negative
60,61,ExpressWay Logistics is unreliable and untrust...,Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [ ]:
print(cs_reviews_df['sentiment'].value_counts())

sentiment
Positive    68
Negative    63
Name: count, dtype: int64


In [ ]:
cs_reviews_df.shape


(131, 3)

In [ ]:
#shows no of columns and rows in the table
rows =cs_reviews_df.shape[0]
print ("Number of rows:",cs_reviews_df.shape[0])
columns = cs_reviews_df.shape[1]
print ("Number of columns:",cs_reviews_df.shape[1])

Number of rows: 131
Number of columns: 3


**(C) Split the Dataset (2 Marks)**

In [ ]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size = 0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

In [ ]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [ ]:
columns_to_select = ['review','sentiment']

In [ ]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [ ]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [ ]:
json.loads(gold_examples)[0]     #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [ ]:
user_message_template = """```{courier_service_review}```"""


**(A) Write Zero Shot System Message (3 Marks)**

In [ ]:
zero_shot_system_message = """Classify customer reviews in the input as positive or negative in sentiment.
Reviews will be delimited by triple backticks, that is,'''.
do not explain your answer your answere should be either Positive or Negative.
"""

# Write zero shot system message here

**(B) Create Zero Shot Prompt (2 Marks)**

In [ ]:
zero_shot_prompt = [
    {"role": "system", "content": zero_shot_system_message},
    {"role": "user", "content": user_message_template.format(courier_service_review=cs_reviews_df)}
]

# Access the 'reviews' column of the Dataset

In [ ]:
zero_shot_prompt

[{'role': 'system',
  'content': "Classify customer reviews in the input as positive or negative in sentiment.\nReviews will be delimited by triple backticks, that is,'''.\ndo not explain your answer your answere should be either Positive or Negative.\n"},
 {'role': 'user',
  'content': "```      id                                             review sentiment\n0      1  ExpressWay Logistics' commitment to transparen...  Positive\n1      2  The tracking system implemented by ExpressWay ...  Positive\n2      3  ExpressWay Logistics is a lifesaver when it co...  Positive\n3      4  Expressway Logistics is the worst courier serv...  Negative\n4      5  ExpressWay Logistics failed to meet my expecta...  Negative\n..   ...                                                ...       ...\n126  127  ExpressWay Logistics has potential, but they n...  Positive\n127  128  While ExpressWay Logistics' pricing is competi...  Negative\n128  129  I had a question about shipping regulations fo...  Positive

In [ ]:
num_tokens_from_messages(zero_shot_prompt)

253

**(C) Write Few Shot System Message (3 Marks)**

In [ ]:
few_shot_system_message = """Classify customer reviews in the input as positive or negative in sentiment.
Reviews will be delimited by triple backticks, that is,'''.
do not explain your answer your answere should be either Positive or Negative.
"""
user_message_template = """```{courier_service_review}```"""

user_input_example1: """
The delivery person was too rude and frustrated me. He was not able to deliver my courier on the promised delivery date.Also when he came to deliver after 4-5 days, he showed no apologies and told me to pick up the courier by myself,even though I paid the service charges and was promised a door to door delivery.I am not happy by this level of service and expected a good service.
"""
Assistant_Output_example1:"""Negative"""

user_input_example2: """
ExpressWay Logistics is a lifesaver when it comes to shipping. Their prompt and professional service ensures that my packages always arrive on time and in excellent condition. What sets them apart is their attention to detail - they take the time to understand my specific needs and tailor their solutions accordingly. I highly recommend ExpressWay Logistics to anyone in need of reliable shipping services.
"""
Assistant_Output_example2:"""Positive"""

user_input_example3:"""
The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.
"""
Assistant_Output_example3:"""Positive"""
new_review = """
I was impressed by the speed of delivery offered by ExpressWay Logistics, as my package arrived sooner than expected. The prompt service exceeded my expectations and demonstrated their commitment to timely deliveries.However, upon receiving the package, I discovered that an item was missing from the shipment. Despite contacting customer service to address the issue, I encountered difficulty in obtaining a resolution, leaving me dissatisfied with the overall experience.
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [ ]:
def create_examples(cs_examples_df, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (cs_examples_df.sentiment == 'Positive')
    negative_reviews = (cs_examples_df.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = cs_examples_df.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = cs_examples_df.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    randomized_examples = examples.sample(2*n, replace=False)
    return randomized_examples.to_json(orient='records')


In [ ]:
examples = create_examples(cs_examples_df, n=4)


**(D) Create Examples For Few shot prompte (2 Marks)**

In [ ]:
# Create Examples
examples = create_examples(cs_examples_df, n=4)


In [ ]:
import json
json_data = json.loads(cs_examples_df.to_json(orient='records'))
print(json_data)


[{'id': 79, 'review': "The packaging used by ExpressWay Logistics for my shipment was inadequate and resulted in damage to the contents. Despite being marked as fragile, the items were poorly packed and insufficiently protected, leading to breakage during transit. The lack of care and attention to detail from ExpressWay Logistics' staff is concerning, and I'm disappointed by their failure to ensure the safe delivery of my belongings.", 'sentiment': 'Negative'}, {'id': 48, 'review': "When planning a high-profile corporate event, logistics are often the last thing on your mind. However, they can make or break the success of the event. That's why I turned to ExpressWay Logistics for assistance with coordinating transportation and logistics for a recent conference.I am more than happy!", 'sentiment': 'Positive'}, {'id': 1, 'review': "ExpressWay Logistics' commitment to transparency gives us confidence in their services. They provide clear and upfront pricing, so we know exactly what to exp

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [ ]:
def create_prompt(few_shot_system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': few_shot_system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [ ]:
few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

In [ ]:
few_shot_prompt

[{'role': 'system',
  'content': "Classify customer reviews in the input as positive or negative in sentiment.\nReviews will be delimited by triple backticks, that is,'''.\ndo not explain your answer your answere should be either Positive or Negative.\n"},
 {'role': 'user',
  'content': '```I was impressed by the speed of delivery offered by ExpressWay Logistics, as my package arrived sooner than expected. The prompt service exceeded my expectations and demonstrated their commitment to timely deliveries.However, upon receiving the package, I discovered that an item was missing from the shipment. Despite contacting customer service to address the issue, I encountered difficulty in obtaining a resolution, leaving me dissatisfied with the overall experience.```'},
 {'role': 'assistant', 'content': 'Negative'},
 {'role': 'user',
  'content': "```ExpressWay Logistics' user-friendly online platform made it easy for me to schedule and track my shipment. The intuitive interface and clear instr

In [ ]:
num_tokens_from_messages(few_shot_prompt)

602

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [ ]:
evaluate_prompt(zero_shot_prompt,gold_examples,user_message_template)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

0.9047619047619048

**(B) Evaluate few shot prompt (2 Marks)**

In [ ]:
evaluate_prompt(few_shot_prompt,gold_examples,user_message_template)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

0.9047619047619048

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [ ]:
zero_shot_performance = []
few_shot_performance = []


In [ ]:
num_eval_runs = 5
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples =create_examples(cs_examples_df)
    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt=create_prompt(few_shot_system_message,examples,user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [02:12<08:50, 132.66s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [04:17<06:24, 128.28s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [08:20<02:03, 123.75s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [10:22<00:00, 124.42s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [ ]:
zero_shot_mean = np.mean(zero_shot_performance)
zero_shot_std = np.std(zero_shot_performance)
# Calculate for Zero Shot
print("Zero Shot:")
print("Mean:", zero_shot_mean)
print("Standard Deviation:", zero_shot_std)

Zero Shot:
Mean: 0.9047619047619048
Standard Deviation: 0.0


In [ ]:
few_shot_mean = np.mean(few_shot_performance)
few_shot_std = np.std(few_shot_performance)
# Calculate for Few Shot
print("\nFew Shot:")
print("Mean:", few_shot_mean)
print("Standard Deviation:", few_shot_std)


Few Shot:
Mean: 0.9047619047619048
Standard Deviation: 0.0


##**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified. )


**Observation, Insights, and Business Perspective**

Preliminary analysis of the "courier-service_reviews.csv" dataset suggests the following:

**Sentiment Analysis observations:**
Total Reviews: 131
Positive Reviews: 68
Negative Reviews: 63
Percentage of Positive Reviews: (68 / 131) * 100 ≈ 51.91%
Percentage of Negative Reviews: (63 / 131) * 100 ≈ 48.09%
**Note: **
Approximately 51.91% of the reviews are positive, and 48.09% are negative.

**Key points observed in Negative Reviews:**
1.Delayed deliveries
2.Damaged packages
3.Poor customer service
4.Inaccurate tracking information

**Business problem and complexity **
Delivery Timeliness: A significant portion of negative reviews highlight delays in deliveries. ExpressWay Logistics should prioritize improving its delivery timeframes, possibly by optimizing routes, investing in additional resources, or implementing real-time tracking systems.

**Package Handling:** Complaints about damaged packages indicate issues with packaging materials, handling procedures, or transportation methods. Implementing stricter quality control measures and investing in protective packaging can resolve these problems.

**Customer Service:** Negative feedback related to customer service suggests a need for improved training and support for customer service representatives. Providing clear communication channels and timely responses to customer inquiries can enhance satisfaction.
Tracking Accuracy: Inaccurate tracking information can lead to customer frustration. Implementing more reliable tracking systems and providing real-time updates can improve transparency and trust.

**Business Use Case:**
**This sentiment analysis results can be used as below areas for improvement:**

**Prioritize Improvement Areas**:
1.Identify the most pressing issues based on the frequency of negative reviews.
2.Measure the Impact of Changes: Track changes in sentiment over time to assess the effectiveness of

Solutioning for the Improvements:
1. Tailor Customer Service: Provide targeted assistance to customers based on their specific concerns.
2.Optimize Operations: Make data-driven decisions to improve efficiency and reduce costs.

Conclusion:
By addressing all the above identified issues and leveraging the insights and Implementing the solutioning from sentiment analysis, ExpressWay Logistics can enhance customer satisfaction, improve its reputation in the courier service market Industry.











